# 1. Documentacion de Referencia

## 1.1 NSIDC data

* NSIDC Home https://nsidc.org/home

* Explore web services, interactive tutorials, and other tools to access and work with NSIDC data https://nsidc.org/data/user-resources/data-tools

* Programmatic Data Access Guide https://nsidc.org/data/user-resources/help-center/programmatic-data-access-guide

<!-- * NSIDC-Data-Access-Notebook https://github.com/nsidc/NSIDC-Data-Access-Notebook -->

## 1.2 SMAP

* Earth - CMR Search - SMAP https://cmr.earthdata.nasa.gov/search/concepts/C2136471727-NSIDC_ECS.html

* NSICD - SMAP https://nsidc.org/data/spl3smp_e/versions/5


* Interfaz grafica web https://worldview.earthdata.nasa.gov/?v=-207.78934227214495,-113.17005215620739,139.7262447856823,55.3388080265203&l=SMAP_L3_Passive_Enhanced_Day_Soil_Moisture(hidden),BlueMarble_NextGeneration(hidden),MODIS_Terra_CorrectedReflectance_TrueColor&lg=false&t=2015-08-08-T16%3A00%3A00Z



In [144]:
#Import packages

import time
from datetime import datetime
import inspect

from utils import load_secrets
from utils import temporal_range
from utils import request_urls_list
from utils import create_root_folder
from utils import create_folder
from utils import delete_folder
from utils import dict_folders
from utils import create_logs_file
from utils import write_log_message
from utils import unzip
from utils import move_images
from utils import execute_request
from utils import check_execution_result

import parameters
from parameters import *

## ETAPA N°0: create_logs_file()

In [145]:
execution_time_start = time.time()
list_parameters = [v for v in dir(parameters) if v[:2] != "__"]
parameters_values = dict(inspect.getmembers(parameters))

In [146]:
datetime_execution = datetime.today().isoformat('T','seconds')
time_now = datetime_execution.replace(':','-')
name_file_log = f'log_ejecucion_{time_now}_{short_name}_{version}_{start_date}_{end_date}.txt'
status_code_log_file,status_message_log_file,log = create_logs_file(name_file_log)

In [147]:
message = f'ATENCION! --> SE INICIA LA EJECUCION DEL PROCESO [{short_name}] CON FECHA: [{datetime_execution}]'
write_log_message(log,'#'*len(message))
write_log_message(log,message)
write_log_message(log,'#'*len(message))

message = f'LOS PARAMETROS A UTILIZAR QUE SE ENCUENTRAN EN EL SCRIPT parameters.py SON:'
write_log_message(log,'#'*150)
write_log_message(log,message)
for parameter in list_parameters:
    valor = parameters_values[parameter]
    message = f'\t{parameter} = {valor}'
    write_log_message(log,message)
write_log_message(log,'#'*150)

## ETAPA N°1: load_secrets()

In [148]:
username,password,email = load_secrets()

In [149]:
status_code_load_secrets = 'OK'
status_message_load_secrets = ''

In [150]:
message = f'#ETAPA N°1: Se cargan los secretos para realizar la conexion al microservicio (API) de EARTHDATA --> load_secrets() --> Estado:[{status_code_load_secrets}] --> Mensaje:[{status_message_load_secrets}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°2: temporal_range()

In [151]:
status_code_time,status_message_time,time_range = temporal_range(start_date,start_time,end_date,end_time)

In [152]:
message = f'#ETAPA N°2: Se construye la ventana temporal [{time_range}] a utilizar en el request --> temporal_range() --> Estado:[{status_code_time}] --> Mensaje:[{status_message_time}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°3: request_urls_list()

In [153]:
#Listado de parametros a incluir en el Request
param_dict = {'short_name': short_name, 
              'version': version,
              'format': formato,
              'time': time_range,
              'Coverage': '',
              'projection': projection,
              'bbox': bbox,
              'email': email,
              'page_size': page_size,
              'request_mode': request_mode
             }

status_code_request_list,status_message_request_list,request_list = request_urls_list(param_dict,coverages)

In [154]:
message = f'#ETAPA N°3: Se obtiene el listado de URLs ({len(request_list)}) a utilizar en el request --> request_urls_list() --> Estado:[{status_code_request_list}] --> Mensaje:[{status_message_request_list}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°4: create_root_folder()

In [155]:
status_code_root_folder,status_message_root_folder,_ = create_root_folder(folder_name_list)

In [156]:
message = f'#ETAPA N°4: Se crean los directorios {folder_name_list} necesarios para la ejecucion --> create_root_folder() --> Estado:[{status_code_root_folder}] --> Mensaje:[{status_message_root_folder}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°5: dict_folders()

In [157]:
status_code_dict_folders,status_message_dict_folders,dict_folder_list = dict_folders(folder_name_list)

In [158]:
message = f'#ETAPA N°5: Se crea un diccionario con los directorios creados en ETAPA N°4 --> dict_folders() --> Estado:[{status_code_dict_folders}] --> Mensaje:[{status_message_dict_folders}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°6: create_folder()

In [159]:
name_folder = f'{time_now}_{short_name}_{version}_{start_date}_{end_date}'
origin_folder = dict_folder_list['path_OUTPUT']
status_code_create_folder,status_message_create_folder,destination_folder = create_folder(origin_folder,name_folder)
message = f'#ETAPA N°6: Se crea el directorio [{name_folder}] dentro del directorio [{origin_folder}] donde se almacena la salida del proceso --> create_folder() --> Estado:[{status_code_create_folder}] --> Mensaje:[{status_message_create_folder}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°7: request()

In [160]:
destination_path = dict_folder_list['path_TMP']
for indice_url,url in enumerate(request_list):
    status_code_execute_request,status_message_execute_request,_ = execute_request(username,password,url,destination_path,num_retries,http_status_list)
    message = f'#ETAPA N°7-{indice_url+1}: Se realiza el request a [{url}] y el resultado se guarda en [{destination_path}] --> execute_request() --> Estado:[{status_code_execute_request}] --> Mensaje:[{status_message_execute_request}] --> {round(time.time() - execution_time_start,4)} seg'
    write_log_message(log,message)

## ETAPA N°8: unzip()

In [161]:
status_code_unzip,status_message_unzip,_ = unzip(dict_folder_list['path_TMP'])

In [162]:
message = f'#ETAPA N°8: Se descomprimen las imagenes obtenidas en la ETAPA N°7 que se guardaron en [{destination_path}] --> unzip() --> Estado:[{status_code_unzip}] --> Mensaje:[{status_message_unzip}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°9: move_images()

In [163]:
status_code_move_images,status_message_move_images,_ = move_images(dict_folder_list['path_TMP'],destination_folder)

In [164]:
message = f'#ETAPA N°9: Se mueven las imagenes desde [{origin_folder}] a [{destination_folder}] --> move_images() --> Estado:[{status_code_move_images}] --> Mensaje:[{status_message_move_images}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°10: delete_folder()

In [165]:
status_code_delete_folder,status_message_delete_folder,_ = delete_folder(dict_folder_list['path_TMP'])

In [166]:
message = f'#ETAPA N°10: Se elimina el directorio [{destination_path}] --> delete_folder() --> Estado:[{status_code_delete_folder}] --> Mensaje:[{status_message_delete_folder}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

## ETAPA N°11: delete_folder()

In [167]:
status_code_check_execution,status_message_check_execution,_ = check_execution_result(destination_folder,start_date,end_date,coverages)

In [168]:
message = f'#ETAPA N°11: Se chequea la cantidad de imagenes obtenidas con respecto a la cantidad teorica --> check_execution_result() --> Estado:[{status_code_check_execution}] --> Mensaje:[{status_message_check_execution}] --> {round(time.time() - execution_time_start,4)} seg'
write_log_message(log,message)

In [169]:
message = f'#SE DA POR FINALIZADO EL PROCESO! --> Las imagenes se encuentran en [{destination_folder}]'
write_log_message(log,message)

In [170]:
log.close()

## main()

In [ ]:
def download_images():
    
    #Request - Listado de parametros
    param_dict = {'short_name': short_name, 
                  'version': version,
                  'format': formato,
                  'time': '',
                  'Coverage': '',
                  'projection': projection,
                  'bbox': bbox,
                  'email': '',
                  'page_size': page_size,
                  'request_mode': request_mode
                 }
    
    #ETAPA N°0: Se crea el archivo donde se guardaran los logs de la ejecucion en curso
    time_now = datetime.today().isoformat('T','seconds').replace(':','-')
    name_file_log = f'log_{short_name}_{version}_{start_date}_{end_date}_Ejecucion_{time_now}.txt'
    status_code_log_file,status_message_log_file,log = create_logs_file(name_file_log)
    
    if status_code_log_file == 'OK':
        #ETAPA N°1: Se cargan los secretos para realizar la conexion al microservicio (API) de EARTHDATA --> load_secrets()
        username,password,email = load_secrets()
        
        if (username is not None) and (password is not None) and (email is not None):
            #Se incorpora 'email' al diccionario de parametros
            param_dict['email'] = email
            #ETAPA N°2: Se construye la ventana temporal a utilizar en el request --> temporal_range()
            status_code_time,status_message_time,time_range = temporal_range(start_date,start_time,end_date,end_time)
            #Se incorpora 'time' al diccionario de parametros
            param_dict['time'] = time_range
            
            if status_code_time == 'OK':
                #ETAPA N°3: Se obtiene el listado de URL's a utilizar en el request --> request_urls_list()
                status_code_request_list,status_message_request_list,request_list = request_urls_list(param_dict,coverages)
                
                if status_code_request == 'OK':
                    #ETAPA N°4: Se crean los directorios necesarios para la ejecucion --> create_folder()
                    status_code_create_folder,status_message_create_folder = create_folder(folder_name_list)
                    
                    if status_code_create_folder == 'OK':
                        #ETAPA N°5: Se incorpora en un diccionario los directorios creados en ETAPA N°4 --> dict_folders()
                        status_code_dict_folders,status_message_dict_folders,dict_folder_list = dict_folders(folder_name_list)
                        
                        if status_code_dict_folders == 'OK':



                        else:
                            return status_code_dict_folders,status_message_dict_folders,dict_folder_list
                        
                    else:
                        return status_code_create_folder,status_message_create_folder,None
                    
                else:
                    return status_code_request_list,status_message_request_list,request_list
                
            else:
                return status_code_time,status_message_time,time_range
            
        else:
            status_code_load_secrets = 'NOK'
            status_message_load_secrets = 'load_secrets() --> Error: No fue posible leer los secretos para acceder al microservicio (API) de EARTHDATA'
            return status_code_load_secrets,status_message_load_secrets,None
        
    else:
        return status_code_log_file,status_message_log_file,_